In [1]:
import base64
import base64
from pprint import pprint

from aes import *


# testing the function
message = b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'  # this is your message in bytes
padded_message = PKCS7_padding(base64.b64decode(message), 16)
print(padded_message)
print(len(padded_message))


b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x06\x06\x06\x06\x06\x06"
144


In [2]:
160-144

16

In [3]:
unknown_string = b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'  # this is your message in bytes
key = b'dTP\xdc"\xb8\xa4\x12\xe3H\xfc\xd5zS\x02\xf2'
# print(base64.b64decode(message))
# print(f"the message is {len(base64.b64decode(message))} bytes long")


In [4]:
#3. Knowing the block size, craft an input block that is exactly 1 byte short 
# (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about 
# what the oracle function is going to put in that last byte position.
res = aes_128_ecb_append(b"AAAAAAAAAAAAAAAA", base64.b64decode(unknown_string), key) #input of 16 bytes
a = get_blocks_in_list(res,16)
pprint(a[0])

res = aes_128_ecb_append(b"AAAAAAAAAAAAAAA", base64.b64decode(unknown_string), key) #input of 15 bytes
a = get_blocks_in_list(res,16)
pprint(a[0])
print(f"len of a[0]: {len(a[0])}")
print( ord(chr(a[0][-15])))
print( a[0][-1]) #first byte of the unknown string. Enc(R)

b'\xa4/\xd8\x8e\xdf\xe6I\x11\x9dCc0\x8d\xbf\xfe\x01'
b'\xf1\xeb\xd1\x96\xc3\x98\x11\xed\xce \x9b\xff\x97w\xecm'
len of a[0]: 16
235
109


In [5]:
def print_each_byte(data):
    for byte in data:
        print(byte, hex(byte), chr(byte))


# testing the function
data = b'\xf1\xeb\xd1\x96\xc3\x98\x11\xed\xce \x9b\xff\x97w\xecm'
data = b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x06\x06\x06\x06\x06\x06"
print_each_byte(data)


82 0x52 R
111 0x6f o
108 0x6c l
108 0x6c l
105 0x69 i
110 0x6e n
39 0x27 '
32 0x20  
105 0x69 i
110 0x6e n
32 0x20  
109 0x6d m
121 0x79 y
32 0x20  
53 0x35 5
46 0x2e .
48 0x30 0
10 0xa 

87 0x57 W
105 0x69 i
116 0x74 t
104 0x68 h
32 0x20  
109 0x6d m
121 0x79 y
32 0x20  
114 0x72 r
97 0x61 a
103 0x67 g
45 0x2d -
116 0x74 t
111 0x6f o
112 0x70 p
32 0x20  
100 0x64 d
111 0x6f o
119 0x77 w
110 0x6e n
32 0x20  
115 0x73 s
111 0x6f o
32 0x20  
109 0x6d m
121 0x79 y
32 0x20  
104 0x68 h
97 0x61 a
105 0x69 i
114 0x72 r
32 0x20  
99 0x63 c
97 0x61 a
110 0x6e n
32 0x20  
98 0x62 b
108 0x6c l
111 0x6f o
119 0x77 w
10 0xa 

84 0x54 T
104 0x68 h
101 0x65 e
32 0x20  
103 0x67 g
105 0x69 i
114 0x72 r
108 0x6c l
105 0x69 i
101 0x65 e
115 0x73 s
32 0x20  
111 0x6f o
110 0x6e n
32 0x20  
115 0x73 s
116 0x74 t
97 0x61 a
110 0x6e n
100 0x64 d
98 0x62 b
121 0x79 y
32 0x20  
119 0x77 w
97 0x61 a
118 0x76 v
105 0x69 i
110 0x6e n
103 0x67 g
32 0x20  
106 0x6a j
117 0x75 u
115 0x73 s
116 0x74 t
32 0x20  
116

In [6]:
# Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance,
# "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.



def find_one_byte_of_unknown_string(idx, probable_pt, block_number, already_decryped_block):  

    if block_number == 0:
        res2 = aes_128_ecb_append(already_decryped_block+ b"AAAAAAAAAAAAAAAA"[0:-idx], base64.b64decode(unknown_string)[block_number*16:], key) # only 15 bytes
    elif block_number !=0:
        res2 = aes_128_ecb_append(already_decryped_block + b"AAAAAAAAAAAAAAAA"[0:-idx], base64.b64decode(unknown_string)[block_number*16:], key) # only 15 bytes

    a2 = get_blocks_in_list(res2,16)
    print(a2)
    last_ct = a2[block_number][-1]
    print(f"last_ct: {last_ct}, hex(last_ct): {hex(last_ct)}")

    #iterate over the last byte of the unknown string from a-z and A-Z and store the result in a dictionary
    for i in range(32, 126):
        # print(i)
        
         
        payload = already_decryped_block+ b"AAAAAAAAAAAAAAAA"[0:-idx] + probable_pt + bytes([i])
        print(f"I'm encrypting: {payload}") 
        testing_ct = aes_128_ecb_append(already_decryped_block+ b"AAAAAAAAAAAAAAAA"[0:-idx] + probable_pt + bytes([i]), base64.b64decode(unknown_string)[block_number*16:], key)
        
        
        test1 = get_blocks_in_list(testing_ct,16)
        test2 = test1[block_number][-1]

        if test2 == last_ct:
            # print(f"test2: {test2}, hex(test2): {hex(test2)}")
            # print(bytes([i]))
            print(f"last byte of unknown string: {i}, chr(i):{chr(i)}, hex(i):{hex(i)}, ord(chr(i)):{ord(chr(i))}")
            # probable_pt = bytes([i])
            break
    return bytes([i])


def find_one_block(prev_decrypted_blocks, block_number):
    probable_pt_concat = b"" 
    for i in range(1,17):
        probable_pt = find_one_byte_of_unknown_string(i, probable_pt_concat, block_number, prev_decrypted_blocks)
        probable_pt_concat += probable_pt
    return probable_pt_concat


block1 = find_one_block(b"", 0)


        

[b'\xf1\xeb\xd1\x96\xc3\x98\x11\xed\xce \x9b\xff\x97w\xecm', b'\x8aC7\xe3rg\x90\x98\x00=\xd9=\x1a\x95\xabA', b'\xb8<\x97\x84\x11\x98\xbb\x98\xac\x13\x0c\xef\x86\xbc\xca\n', b'P\x12\x99;\x89\x87|\x04\xfc\x80i\xf5\xa9\xde\xe4\x11', b'\x11Z9\x0bT\x1dA\xecZ\x95\x9bpChW\x8f', b'n\xbbb\x9a\xf1\xfb\x0fx\xa8\xcc6\x97\x90\x9fG\x04', b'I\xe2^\\\xdb6\x14\xc3\xfeC\xcf@\xbc\x93\xb1\x1e', b'\x9aff\x82\xe7+\x1f\xd8\xee\xe7\xaalS\x95(L', b'\x02\xc4X\xd2\xfer\xec\xa1\x92\x12\xe3\xfe\x1fQ6\xfb', b'*-\xdbE)\xfd\xfbMv5z\xcfy\xd2\xb2$']
last_ct: 109, hex(last_ct): 0x6d
I'm encrypting: b'AAAAAAAAAAAAAAA '
I'm encrypting: b'AAAAAAAAAAAAAAA!'
I'm encrypting: b'AAAAAAAAAAAAAAA"'
I'm encrypting: b'AAAAAAAAAAAAAAA#'
I'm encrypting: b'AAAAAAAAAAAAAAA$'
I'm encrypting: b'AAAAAAAAAAAAAAA%'
I'm encrypting: b'AAAAAAAAAAAAAAA&'
I'm encrypting: b"AAAAAAAAAAAAAAA'"
I'm encrypting: b'AAAAAAAAAAAAAAA('
I'm encrypting: b'AAAAAAAAAAAAAAA)'
I'm encrypting: b'AAAAAAAAAAAAAAA*'
I'm encrypting: b'AAAAAAAAAAAAAAA+'
I'm encryptin

In [7]:
print(f"block1: {block1}")

print(f"len of block1: {len(block1)}")
find_one_block(block1, 1)

block1: b"Rollin' in my 5."
len of block1: 16
[b'\x14\x0fno\xa0.&\xb2\x95)\x08\x85_t/I', b'=4\x8a\x85/\xdb-\xaf\x9e\x9e\x11R\xd5\xa5\x8c\x9d', b'\xb8<\x97\x84\x11\x98\xbb\x98\xac\x13\x0c\xef\x86\xbc\xca\n', b'P\x12\x99;\x89\x87|\x04\xfc\x80i\xf5\xa9\xde\xe4\x11', b'\x11Z9\x0bT\x1dA\xecZ\x95\x9bpChW\x8f', b'n\xbbb\x9a\xf1\xfb\x0fx\xa8\xcc6\x97\x90\x9fG\x04', b'I\xe2^\\\xdb6\x14\xc3\xfeC\xcf@\xbc\x93\xb1\x1e', b'\x9aff\x82\xe7+\x1f\xd8\xee\xe7\xaalS\x95(L', b'\x02\xc4X\xd2\xfer\xec\xa1\x92\x12\xe3\xfe\x1fQ6\xfb', b'*-\xdbE)\xfd\xfbMv5z\xcfy\xd2\xb2$']
last_ct: 157, hex(last_ct): 0x9d
I'm encrypting: b"Rollin' in my 5.AAAAAAAAAAAAAAA "
I'm encrypting: b"Rollin' in my 5.AAAAAAAAAAAAAAA!"
I'm encrypting: b'Rollin\' in my 5.AAAAAAAAAAAAAAA"'
I'm encrypting: b"Rollin' in my 5.AAAAAAAAAAAAAAA#"
I'm encrypting: b"Rollin' in my 5.AAAAAAAAAAAAAAA$"
I'm encrypting: b"Rollin' in my 5.AAAAAAAAAAAAAAA%"
I'm encrypting: b"Rollin' in my 5.AAAAAAAAAAAAAAA&"
I'm encrypting: b"Rollin' in my 5.AAAAAAAAAAAA

b'0}}}}B}Ug}}}}}})'

In [8]:
print(hex(82))
print(hex(181))

0x52
0xb5


In [9]:
b"AAA"[0:-2]

b'A'

In [10]:
a=b"12345678"

print(a[4:])

b'5678'


In [11]:
for i in range(256):
    print(i, chr(i), hex(i))

0   0x0
1  0x1
2  0x2
3  0x3
4  0x4
5  0x5
6  0x6
7  0x7
8 0x8
9 	 0x9
10 
 0xa
11  0xb
12  0xc
 0xd
14  0xe
15  0xf
16  0x10
17  0x11
18  0x12
19  0x13
20  0x14
21  0x15
22  0x16
23  0x17
24  0x18
25  0x19
26  0x1a
27  0x1b
28  0x1c
29  0x1d
30  0x1e
31  0x1f
32   0x20
33 ! 0x21
34 " 0x22
35 # 0x23
36 $ 0x24
37 % 0x25
38 & 0x26
39 ' 0x27
40 ( 0x28
41 ) 0x29
42 * 0x2a
43 + 0x2b
44 , 0x2c
45 - 0x2d
46 . 0x2e
47 / 0x2f
48 0 0x30
49 1 0x31
50 2 0x32
51 3 0x33
52 4 0x34
53 5 0x35
54 6 0x36
55 7 0x37
56 8 0x38
57 9 0x39
58 : 0x3a
59 ; 0x3b
60 < 0x3c
61 = 0x3d
62 > 0x3e
63 ? 0x3f
64 @ 0x40
65 A 0x41
66 B 0x42
67 C 0x43
68 D 0x44
69 E 0x45
70 F 0x46
71 G 0x47
72 H 0x48
73 I 0x49
74 J 0x4a
75 K 0x4b
76 L 0x4c
77 M 0x4d
78 N 0x4e
79 O 0x4f
80 P 0x50
81 Q 0x51
82 R 0x52
83 S 0x53
84 T 0x54
85 U 0x55
86 V 0x56
87 W 0x57
88 X 0x58
89 Y 0x59
90 Z 0x5a
91 [ 0x5b
92 \ 0x5c
93 ] 0x5d
94 ^ 0x5e
95 _ 0x5f
96 ` 0x60
97 a 0x61
98 b 0x62
99 c 0x63
100 d 0x64
101 e 0x65
102 f 0x66